In [4]:
import pandas as pd
matches = pd.read_csv("matches.csv", index_col=0)
matches.head()
matches.shape
matches["team"].value_counts()
matches[matches["team"] == "Liverpool"]
matches["round"].value_counts()
matches.dtypes
matches["date"] = pd.to_datetime(matches["date"])
matches.dtypes
matches["venue_code"] = matches["venue"].astype("category").cat.codes
matches["opp_code"] = matches["opponent"].astype("category").cat.codes
matches["hour"] = matches["time"].str.replace(":.+","",regex=True).astype("int")
matches["day_code"] = matches["date"].dt.dayofweek
matches["target"] = (matches["result"] == "W").astype("int")

from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1)
train = matches[matches["date"] < '2022-01-01']
test = matches[matches["date"] > '2022-01-01']
predictors = ["venue_code", "opp_code", "hour", "day_code"]
rf.fit(train[predictors], train["target"])
preds=rf.predict(test[predictors])

from sklearn.metrics import accuracy_score
acc = accuracy_score(test["target"], preds)
acc
combined = pd.DataFrame(dict(actual=test["target"], prediction=preds))
pd.crosstab(index=combined["actual"], columns=combined["prediction"])

from sklearn.metrics import precision_score
precision_score(test["target"], preds)
grouped_matches = matches.groupby("team")
group = grouped_matches.get_group("Chelsea")

def rolling_averages(group,cols,new_cols):
    group = group.sort_values("date")
    rolling_stats = group[cols].rolling(3, closed="left").mean()
    group[new_cols] = rolling_stats
    group = group.dropna(subset=new_cols)
    return group

cols = ["gf", "ga", "sh", "sot", "dist", "fk", "pk", "pkatt"]
new_cols = [f"{c}_rolling" for c in cols]
new_cols
rolling_averages(group, cols, new_cols)
matches_rolling = matches.groupby("team").apply(lambda x: rolling_averages(x, cols, new_cols))
matches_rolling = matches_rolling.droplevel("team")
matches_rolling.index = range(matches_rolling.shape[0])

def make_predictions(data, predictors):
    train = data[data["date"] < '2022-01-01']
    test = matches[matches["date"] > '2022-01-01']
    rf.fit(train[predictors], train["target"])
    preds=rf.predict(test[predictors])
    combined = pd.DataFrame(dict(actual=test["target"], predicted=preds), index=test.index)
    precision = precision_score(test["target"], preds)
    return combined, precision
combined.merge(matches_rolling[["date", "team", "opponent", "result"]], left_index=True, right_index=True)
combined
class MissingDict(dict):
    __missing__ = lambda self, key: key
map_values = {
    "Brighton and Hove Albion": "Brighton",
    "Manchester United": "Manchester Utd",
    "Newcastle United": "Newcastle Utd",
    "Tottenham Hotspur": "Tottenham",
    "West Ham United": "West Ham",
    "Wolverhampton Wanders": "Wolves"
}
mapping = MissingDict(**map_values)
mapping["West Ham United"]
combined["new_team"] = combined["team"].map(mapping)
combined
merged = combined.merge(combined, left_on=["date", "new_team"], right_on=["date", "opponent"])
merged

C:\Users\shobh\AppData\Local\Temp\ipykernel_15104\704289164.py:47: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  matches_rolling = matches.groupby("team").apply(lambda x: rolling_averages(x, cols, new_cols))


,actual,prediction
31,1,0
32,0,0
34,1,0
35,1,1
37,0,1
...,...,...
33,0,0
34,0,0
35,1,0
36,0,0
